# Creating a text-heavy COVID-19 graphic using data

A few weeks ago, I was inspired by [a graphic by Reuters](https://graphics.reuters.com/world-coronavirus-tracker-and-maps/) showing a simple list of countries, and whether or not their COVID-19 metrics had increased or decreased. I wanted to try doing something similar, but breaking them into categories: those whose cases have gone up, those whose cases have gone down, and those whose cases have not changed vs. a week ago.

As always, let's start by importing pandas, and reading in OWID data directly from the source.

In [229]:
import pandas as pd
pd.options.mode.chained_assignment = None   # This disables a rather annoying but inconsequential error below that I can't seem to fix.

raw = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv')

Next, we need to determine which countries have increased, decreased, or stayed the same compared with two dates. Let's filter and reshape the data, first. We'll also add a new column calculated by subtracting cases on the two dates.

In [230]:
date1 = "2021-01-28"
date2 = "2021-01-14"

data = (raw.loc[raw["date"].isin([date2, date1])]
            .loc[raw["population"] > 1000000]
            .dropna(subset=["continent"])
            .pivot_table(index=["location"], columns="date", values="new_cases_per_million")
            .reset_index()
            .set_index("location")
            .rename_axis(columns="")
          )

data.loc[:,"diff"] = (data[date1] - data[date2]).astype(int)
data = data[["diff"]]

Next, we need to concatenate together a list of all the countries with positive case counts (that is, they're rising).

In [231]:
positives = data.loc[data['diff'].gt(0),:]
positives.loc[:,"text"] = positives.index + " (+" + positives["diff"].astype(str) + ")"

Next, we create an empty dataframe which we'll put our long string of country names and values into.

In [232]:
export = pd.DataFrame({"countries": ["", "", ""]}, index=["Positives", "Negatives", "No change"])
export.loc["Positives", "countries"] = ', '.join(positives["text"].astype(str))

display(export)

,countries
Positives,"Albania (+79), Bahrain (+25), Belgium (+11), B..."
Negatives,
No change,


Now, we repeat in a similar way for negative values, and for those that haven't changed, and add them to our final dataframe.

In [233]:
negatives = data.loc[data['diff'].lt(0),:]
negatives.loc[:,"text"] = negatives.index + " (" + negatives["diff"].astype(str) + ")"
export.loc["Negatives", "countries"] = ', '.join(negatives["text"].astype(str))

no_change = data.loc[data['diff'] == 0, :]
export.loc["No change", "countries"] = ', '.join(no_change.index)

display(export)

,countries
Positives,"Albania (+79), Bahrain (+25), Belgium (+11), B..."
Negatives,"Afghanistan (-1), Angola (-2), Argentina (-83)..."
No change,"Algeria, Australia, Benin, Burundi, Cambodia, ..."


Now we take it to Datawrapper! [Here's the final result](https://www.datawrapper.de/_/nDOb1/), using individual row stylings to colour based on positives or negatives.